In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from gen.curriculum import get_cbc_grouped_questions, get_rubrics_by_sub_strand, get_all_strand_names
from gen.utils import *

In [2]:
names = get_all_strand_names()
print(json.dumps(names))

["Scientific Investigation", "Introduction to Integrated Science", "Laboratory Safety", "Laboratory Apparatus and Instruments", "Mixtures, Elements and Compounds", "Mixtures", "Acids, Bases and Indicators", "Living Things and Their Environment", "Human Reproductive System", "Human Excretory System", "Force and Energy", "Electrical Energy", "Magnetism", "Mixtures, Elements and Compounds", "Elements and Compounds", "Physical and Chemical Changes", "Classes of Fire", "Living Things and the Environment", "The Cell", "Movement of Materials In and Out of the Cell", "Reproduction in Human Beings", "Force and Energy", "Transformation of Energy", "Pressure", "Mixtures, Elements and Compounds", "Structure of the Atom", "Metals and Alloys", "Water Hardness", "Living Things and Their Environment", "Pollution", "Conservation of the Environment", "Force and Energy", "Friction", "Machines"]


## Generate questions

In [5]:
grouped_questions = get_cbc_grouped_questions(
    strand_ids=[1], 
    question_count=5,
    bloom_skill_count=2,
    is_debug=True,
)


✅ Question breakdown written to /Users/melaniefayne/Desktop/mtihani/mtihani_api/mtihaniapi/gen/output/question_breakdown.json. Total: 3


In [6]:
all_question_list = generate_llm_question_list(
    grouped_question_data=grouped_questions,
    is_debug=True,
)

# If there was a generation error
if not isinstance(all_question_list, list):
    print(all_question_list)


Introduction to Integrated Science =========
📝 Input token count (gpt-4o): 686
📦 Raw LLM output:
 content='[{"question":"Amina noticed that her plants grow better when she waters them with rainwater instead of tap water. Using your knowledge of Integrated Science, explain why rainwater might be more beneficial for plant growth.","expected_answer":"Rainwater is often more beneficial for plant growth because it is naturally soft and free of chemicals like chlorine, which are present in tap water. It also contains dissolved nitrogen compounds that can act as a natural fertilizer."},{"question":"Brian is trying to decide whether to use a solar cooker or a charcoal stove for cooking at home. Evaluate the benefits of using a solar cooker over a charcoal stove in terms of environmental impact and cost.","expected_answer":"Using a solar cooker is more environmentally friendly because it does not produce smoke or carbon emissions, unlike a charcoal stove. It also reduces the cost of buying cha

In [7]:
exam_questions = get_db_question_objects(
    all_question_list=all_question_list,
    is_debug=True,
)


✅ Question list to /Users/melaniefayne/Desktop/mtihani/mtihani_api/mtihaniapi/gen/output/question_list.json. Total: 5


## Simulate student answers

In [8]:
def convert_to_single_qa_list(data):
    result = []
    for index, item in enumerate(data):
        questions = item["questions"]
        expected_answers = item["expected_answers"]

        if isinstance(questions, str):
            questions = [questions]
        if isinstance(expected_answers, str):
            expected_answers = [expected_answers]

        if questions and expected_answers:
            result.append({
                "id": index + 1,
                "question": questions[0],
                "expected_answer": expected_answers[0]
            })

    return result

exam_output = convert_to_single_qa_list(exam_questions)
print(exam_output)

[{'id': 1, 'question': 'In the science laboratory, Amina observes that a beaker is used to measure liquids, but she is unsure about its accuracy compared to a graduated cylinder. What is the primary reason a graduated cylinder is preferred for precise measurements?', 'expected_answer': 'A graduated cylinder is preferred for precise measurements because it has more accurate and finer measurement markings than a beaker.'}, {'id': 2, 'question': 'During a science fair, Amina and Brian are tasked with creating a poster that highlights the importance of laboratory safety. What key elements should they include to effectively communicate the significance of safety in the laboratory?', 'expected_answer': 'They should include common hazards and their symbols, causes of accidents, first aid measures, and the benefits of a safe working environment.'}, {'id': 3, 'question': 'Amina noticed that her plants grow better when she waters them with rainwater instead of tap water. Using your knowledge of 

In [9]:
sample_students = [{'id': 1, 'avg_score': 12}, {'id': 2, 'avg_score': 95}, {
    'id': 3, 'avg_score': 54}, {'id': 4, 'avg_score': 71},]

In [10]:
parsed_answers = generate_llm_exam_answers_list(
    llm=OPENAI_LLM_4O,
    exam_data=exam_output,
    student_data=sample_students,
    is_debug=True,
)

📝 Input token count (gpt-4o): 951
📦 Raw LLM output:
 content='```json\n[\n  {\n    "id": 1,\n    "answers": [\n      {\n        "question_id": "1",\n        "answer": "A beaker is not as good as a graduated cylinder because it doesn\'t measure well."\n      },\n      {\n        "question_id": "2",\n        "answer": "They should say don\'t run and wear goggles."\n      },\n      {\n        "question_id": "3",\n        "answer": "Rainwater is better because it is natural."\n      },\n      {\n        "question_id": "4",\n        "answer": "Use a cup with water and see how much it goes up."\n      },\n      {\n        "question_id": "5",\n        "answer": "If you don\'t follow rules, you can get hurt. We should be careful."\n      }\n    ]\n  },\n  {\n    "id": 2,\n    "answers": [\n      {\n        "question_id": "1",\n        "answer": "A graduated cylinder is preferred because it has more precise markings than a beaker, making it more accurate for measuring liquids."\n      },\n     

In [ ]:
# # ==== DB MOCK ANSWERS
# # ====================
# exam = []
# EXAM_FILE = "data/exam.json"

# with open(EXAM_FILE, "r") as f:
#     exam = json.load(f)

# print(exam)


# students = []
# STUDENT_FILE = "data/classroom.json"

# with open(STUDENT_FILE, "r") as f:
#     students = json.load(f)

# print(students)


# parsed_answers = generate_llm_exam_answers_list(
#     llm=OPENAI_LLM_4O,
#     exam_data=exam,
#     student_data=students,
#     is_debug=True,
# )

[{'id': 29, 'question': 'During a science class, Amina noticed a strange smell coming from a chemical spill on the floor. Analyze the situation and identify the potential hazard symbol that should be associated with this spill.', 'expected_answer': "The potential hazard symbol associated with the chemical spill could be the 'corrosive' symbol, indicating that the chemical can cause damage to skin or materials."}, {'id': 30, 'question': 'What color does blue litmus paper turn when dipped into lemon juice?', 'expected_answer': 'Blue litmus paper turns red when dipped into lemon juice, indicating the presence of an acid.'}, {'id': 31, 'question': 'Amina noticed that her voice has started to change and she is growing taller; what are these changes called and why do they occur?', 'expected_answer': 'These changes are called puberty, and they occur due to hormonal changes that prepare the body for adulthood and reproduction.'}, {'id': 32, 'question': 'Amina noticed that when she rides her bi

## Grade answers

In [12]:
def get_answers_for_question(target_question_id, start_count):
    result = []
    count = start_count
    for entry in parsed_answers:
        for answer in entry["answers"]:
            if answer["question_id"] == str(target_question_id):
                result.append({
                    "answer_id": count,
                    "answer": answer["answer"],
                })
                count += 1
    return result, count


grouped_answers_data = []
count = 1

for idx, q in enumerate(exam_questions):
    item = {
        "question": q["description"],
        "expected_answer": q["expected_answer"],
        "rubrics": get_rubrics_by_sub_strand(
            sub_strand_name=q["sub_strand"],
        )
    }
    student_answers, count = get_answers_for_question(idx + 1, count)
    item["student_answers"] = student_answers

    grouped_answers_data.append(item)

In [13]:
ANSWERS_LIST_OUTPUT_FILE = "output/answers_list.json"
with open(ANSWERS_LIST_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(grouped_answers_data, f, ensure_ascii=False, indent=4)
print(f"✅ Mocked Answers list written to {ANSWERS_LIST_OUTPUT_FILE}")

✅ Mocked Answers list written to output/answers_list.json


In [14]:
parsed_grades = generate_llm_answer_grades_list(
    llm=OPENAI_LLM_4O,
    grouped_answers_data=grouped_answers_data,
    is_debug=True,
)


In the science laboratory, Amina observes that a beaker is used to measure liquids, but she is unsure about its accuracy compared to a graduated cylinder. What is the primary reason a graduated cylinder is preferred for precise measurements? =========
📝 Input token count (gpt-4o): 519
📦 Raw LLM output:
 content='```json\n[\n    {"answer_id": 1, "score": 2},\n    {"answer_id": 2, "score": 4},\n    {"answer_id": 3, "score": 3},\n    {"answer_id": 4, "score": 3}\n]\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 526, 'total_tokens': 588, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BXRUzaS4azLq0FyzEXs8d1vmdi0tc', 'service_tier': 'default', 'finish_reason': 'stop', 'logpro

## Create Insights

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from gen.utils import *

In [2]:
# strand_performance_data = [
#     {
#         "strand_name": "Mixtures (G7)",
#         "strand_grade": 7,
#         "avg_score": 74.5,
#         "avg_expectation_level": "Meets",
#         "bloom_skill_scores": [],
#         "score_variance": {"min": 40.0, "max": 99.0, "std_dev": 10.2},
#         "sub_strand_scores": [
#             {"name": "Elements and Compounds", "percentage": 81.2, "strand_difference": 6.7,
#              "strand_difference_descriptor": "Above Strand Average"},
#             {"name": "Physical and Chemical Changes", "percentage": 66.5,
#              "strand_difference": -8.0, "strand_difference_descriptor": "Below Strand Average"}
#         ],
#         "top_students": [
#             {"student_name": "Akinyi", "avg_score": 98.2,
#                 "avg_expectation_level": "Exceeds", "exam_id": 10, "student_id": 3}
#         ],
#         "bottom_students": [
#             {"student_name": "Wanjiru", "avg_score": 44.5,
#                 "avg_expectation_level": "Below", "exam_id": 10, "student_id": 9}
#         ]
#     }
# ]

# insights_data = generate_llm_strand_insights(
#     strand_performance_data=strand_performance_data,
#     is_debug=True,
# )

In [3]:
# sub_strand_correlations = [
#     {
#         "name": "Elements and Compounds",
#         "average_correlation": -0.48,
#         "strongest_negative_pair": "Acids, Bases and Indicators",
#         "correlation": -0.67
#     }
# ]
# insights_data = generate_llm_sub_strand_corr_insights(
#     sub_strand_correlations=sub_strand_correlations,
#     is_debug=True,
# )

In [4]:
class_performance_data = {
    "average_score": 74.1,
    "average_expectation_level": "Meets",
    "score_distribution": "Scores range from 40 to 99, with most students scoring between 70 and 80. Standard deviation is 10.2.",
    "expectation_level_distribution": "Exceeds: 20%, Meets: 60%, Approaching: 15%, Below: 5%",
    "bloom_skill_scores":
    [{"name": "Synthesis", "percentage": 81.67}, {"name": "Knowledge", "percentage": 81.0}, {"name": "Application", "percentage": 73.44}, {
        "name": "Analysis", "percentage": 68.5}, {"name": "Comprehension", "percentage": 71.25}, {"name": "Evaluation", "percentage": 65.0}],
}

insights_data = generate_llm_class_perf_insights(
    class_performance_data=class_performance_data,
    is_debug=True,
)

📝 Input token count (gpt-4o): 443
📦 Raw LLM output:
 content='```json\n[\n    "The average score of 74.1% indicates that the class is generally performing at a satisfactory level, with most students meeting expectations.",\n    "The majority of students fall into the \'Meets\' expectation level, accounting for 60% of the class, which aligns with the average expectation level.",\n    "A notable strength is the high performance in \'Synthesis\' and \'Knowledge\' skills, with scores of 81.67% and 81.0% respectively, suggesting strong capabilities in these areas.",\n    "The \'Evaluation\' skill shows the lowest performance at 65.0%, indicating a potential area for improvement in critical thinking and judgment.",\n    "The score distribution reveals a wide range of performance, from 40 to 99, with a standard deviation of 10.2, suggesting variability in student understanding and mastery.",\n    "A small percentage of students (5%) are performing \'Below\' expectations, which may require tar

In [5]:
INSIGHTS_OUTPUT_FILE = "output/insights_lits.json"
with open(INSIGHTS_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(insights_data, f, ensure_ascii=False, indent=4)
print(f"✅ Insights written to {INSIGHTS_OUTPUT_FILE}")

✅ Insights written to output/insights_lits.json
